# **Build a RAG System on “Leave No Context Behind” Paper**

In [1]:
pip install pypdf

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install langchain_google_genai

  Using cached langchain_google_genai-1.0.2-py3-none-any.whl (28 kB)
  Using cached google_generativeai-0.5.2-py3-none-any.whl (146 kB)
  Using cached langchain_core-0.1.45-py3-none-any.whl (291 kB)
  Using cached google_api_python_client-2.127.0-py2.py3-none-any.whl (12.7 MB)
  Using cached google_ai_generativelanguage-0.6.2-py3-none-any.whl (664 kB)
  Using cached google_auth-2.29.0-py2.py3-none-any.whl (189 kB)
  Using cached google_api_core-2.18.0-py3-none-any.whl (138 kB)
  Using cached langsmith-0.1.50-py3-none-any.whl (115 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl (12 kB)
  Using cached googleapis_common_protos-1.63.0-py2.py3-none-any.whl (229 kB)
  Using cached google_auth_httplib2-0.2.0-py2.py3-none-any.whl (9.3 kB)
  Using cached httplib2-0.22.0-py3-none-any.whl (96 kB)
  Using cached grpcio-1.62.2-cp310-cp310-win_amd64.whl (3.8 MB)
  Using cached grpcio_status-1.62.2-py3-none-any.whl (14 kB)
  Attempting uninstall: jsonpatch
    Found existing installation: json

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

# Setup API Key
f = open(r"C:\Users\HP\OneDrive\Desktop\projects\Search_Engine\keys\Lang_Chain_Key.txt")
GOOGLE_API_KEY = f.read()

chat_model = ChatGoogleGenerativeAI(google_api_key=GOOGLE_API_KEY, model="gemini-1.5-pro-latest")

# Loading the Document

In [4]:
pip install --upgrade langchain

     -------------------------------------- 817.7/817.7 kB 2.5 MB/s eta 0:00:00
     ---------------------------------------- 1.9/1.9 MB 3.5 MB/s eta 0:00:00
     -------------------------------------- 370.7/370.7 kB 1.5 MB/s eta 0:00:00
     -------------------------------------- 50.4/50.4 kB 287.1 kB/s eta 0:00:00
     -------------------------------------- 76.4/76.4 kB 384.7 kB/s eta 0:00:00
     -------------------------------------- 49.4/49.4 kB 358.9 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [8]:
# Load a document

from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(r"C:\Users\HP\OneDrive\Desktop\projects\Search_Engine\2404.07143.pdf")

data = loader.load_and_split()

data[:5]

[Document(page_content='Preprint. Under review.\nLeave No Context Behind:\nEfficient Infinite Context Transformers with Infini-attention\nTsendsuren Munkhdalai, Manaal Faruqui and Siddharth Gopal\nGoogle\ntsendsuren@google.com\nAbstract\nThis work introduces an efficient method to scale Transformer-based Large\nLanguage Models (LLMs) to infinitely long inputs with bounded memory\nand computation. A key component in our proposed approach is a new at-\ntention technique dubbed Infini-attention. The Infini-attention incorporates\na compressive memory into the vanilla attention mechanism and builds\nin both masked local attention and long-term linear attention mechanisms\nin a single Transformer block. We demonstrate the effectiveness of our\napproach on long-context language modeling benchmarks, 1M sequence\nlength passkey context block retrieval and 500K length book summarization\ntasks with 1B and 8B LLMs. Our approach introduces minimal bounded\nmemory parameters and enables fast strea

# Spliting the document into chunks

In [10]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [11]:
# Spliting the document into chunks
from langchain_text_splitters import NLTKTextSplitter

text_splitter = NLTKTextSplitter(chunk_size=500, chunk_overlap=100)

chunks = text_splitter.split_documents(data)

print(len(chunks))

Created a chunk of size 568, which is longer than the specified 500
Created a chunk of size 506, which is longer than the specified 500
Created a chunk of size 633, which is longer than the specified 500


110


# Creating Chunk Embedding

In [12]:
# Creating Chunks Embedding
# We are just loading OpenAIEmbeddings

from langchain_google_genai import GoogleGenerativeAIEmbeddings

embedding_model = GoogleGenerativeAIEmbeddings(google_api_key=GOOGLE_API_KEY, model="models/embedding-001")

# vectors = embeddings.embed_documents(chunks)

# Storing the chunks in vector

In [14]:
pip install chromadb


     ------------------------------------ 526.8/526.8 kB 786.3 kB/s eta 0:00:00
     ------------------------------------ 158.3/158.3 kB 949.1 kB/s eta 0:00:00
     ---------------------------------------- 2.2/2.2 MB 3.1 MB/s eta 0:00:00
     -------------------------------------- 106.1/106.1 kB 1.2 MB/s eta 0:00:00
     ---------------------------------------- 5.6/5.6 MB 13.8 MB/s eta 0:00:00
     -------------------------------------- 78.3/78.3 kB 618.7 kB/s eta 0:00:00
     --------------------------------------- 60.1/60.1 kB 26.8 kB/s eta 0:00:00
     -------------------------------------- 60.8/60.8 kB 648.4 kB/s eta 0:00:00
     --------------------------------------- 47.2/47.2 kB 40.1 kB/s eta 0:00:00
     --------------------------------------- 41.3/41.3 kB 13.5 kB/s eta 0:00:00
     ---------------------------------------- 1.6/1.6 MB 5.1 MB/s eta 0:00:00
     -------------------------------------- 91.9/91.9 kB 523.3 kB/s eta 0:00:00
     -------------------------------------- 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.24.0 requires tokenizers!=0.11.3,<0.14,>=0.11.1, but you have tokenizers 0.19.1 which is incompatible.


In [15]:
# Store the chunks in vector store
from langchain_community.vectorstores import Chroma

# Embed each chunk and load it into the vector store
db = Chroma.from_documents(chunks, embedding_model, persist_directory="Downloads")

# Persist the database on drive
db.persist()

In [16]:
# Setting a Connection with the ChromaDB
connection = Chroma(persist_directory="Downloads", embedding_function=embedding_model)

# Settingup the Vector Store as a Retriever

In [17]:
# Converting CHROMA db_connection to Retriever Object
retriever = connection.as_retriever(search_kwargs={"k": 5})

print(type(retriever))

<class 'langchain_core.vectorstores.VectorStoreRetriever'>


Now let’s write the actual application logic. We want to create a simple application that takes a user question, searches for documents relevant to that question, passes the retrieved documents and initial question to a model, and returns an answer.

# Based on users query retrieving the context

### Query -1

In [18]:
user_query = "What is LLMs?"

In [19]:
retrieved_docs = retriever.invoke(user_query)

In [20]:
len(retrieved_docs)

5

In [21]:
print(retrieved_docs[0].page_content)

However, the LLMs in their current state
have yet to see an effective, practical compres-
sive memory technique that balances simplicity along with quality.

1arXiv:2404.07143v1  [cs.CL]  10 Apr 2024


### Query - 2

In [22]:
user_query = "Tell me about LLMs?"

In [23]:
retrieved_docs = retriever.invoke(user_query)

In [24]:
len(retrieved_docs)

5

In [25]:
print(retrieved_docs[0].page_content)

However, the LLMs in their current state
have yet to see an effective, practical compres-
sive memory technique that balances simplicity along with quality.

1arXiv:2404.07143v1  [cs.CL]  10 Apr 2024


# Passing the context and questioning to the LLM

In [26]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

chat_template = ChatPromptTemplate.from_messages([
    # System Message Prompt Template
    SystemMessage(content="""You are a Helpful AI Bot.
    You take the context and question from user. Your answer should be based on the specific context."""),
    # Human Message Prompt Template
    HumanMessagePromptTemplate.from_template("""Answer the question based on the given context.
    Context:
    {context}
    Question:
    {question}

    Answer: """)
])

In [27]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [28]:
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | chat_template
    | chat_model
    | output_parser
)

## Query - 1

In [29]:
from IPython.display import Markdown as markdown
response = rag_chain.invoke("What is Scaled Dot-product Attention?")

markdown(response)

## Scaled Dot-Product Attention Explained

Unfortunately, the provided text doesn't explicitly define "Scaled Dot-Product Attention." However, based on the context and references, we can infer some key points:

**Dot-Product Attention:**

* This is a fundamental attention mechanism where the attention weights are calculated using the dot product between query and key vectors. 
* The context mentions that it's used within each segment to compute the standard causal attention.
* References like "Efficient attention: Attention with linear complexities" (Shen et al., 2018) likely delve deeper into the specifics of its calculation and efficiency improvements.

**Scaling:**

* The term "scaled" implies a normalization step is involved. This is typically done by dividing the dot product by the square root of the dimension of the key vectors. 
* This scaling helps to prevent the softmax function (often used in attention mechanisms) from having extremely small gradients, which can hinder training.

**Contextual Clues:**

* The paper contrasts Scaled Dot-Product Attention with "local attention" (Dai et al., 2019) which discards previous segment information. This suggests Scaled Dot-Product Attention might have a broader context window or a mechanism to incorporate information from previous segments.
* The proposed "Infini-attention" is described as having minimal changes to the standard scaled dot-product attention. This further implies it's a well-known and established mechanism. 

**Further Exploration:**

To fully understand Scaled Dot-Product Attention, I recommend:

* **Consulting the referenced papers:** Especially "Efficient attention: Attention with linear complexities" (Shen et al., 2018) for a deeper understanding of its mechanics and efficiency considerations.
* **Exploring resources on attention mechanisms:** Many online resources and textbooks provide detailed explanations of Scaled Dot-Product Attention and other attention variants.
* **Examining the full paper:**  Once available, the full paper associated with this context will likely provide a more comprehensive explanation of Scaled Dot-Product Attention within the context of "Infini-attention" and its proposed advancements. 


## Query - 2

In [30]:
response = rag_chain.invoke("Memory and Effective Context Window?")

markdown(response)

## Memory and Effective Context Window: Balancing Power and Efficiency

The provided text discusses the challenge of balancing memory requirements with the need for a large effective context window in LLMs (Large Language Models).  Here's a breakdown:

**The Problem:**

* LLMs benefit from access to a large context window, meaning they can "remember" and utilize information from a longer sequence of previous text.
* However, storing and processing this information can be computationally expensive and memory-intensive, especially as the context window grows.

**Existing Approaches:**

The text mentions several approaches to address this challenge:

* **Segment-level memory models:** These models divide the context into segments and selectively store and access them. However, they still face limitations in terms of efficiency and effectiveness.
* **System-level optimization:** This involves leveraging specific hardware architectures to improve the efficiency of attention calculations, which are crucial for utilizing the context window.
* **Memorizing Transformers:** This approach stores the entire Key-Value (KV) states as context but restricts contextual computation to a single layer to manage storage costs. It also utilizes a kNN retriever to build a context window covering the entire sequence history.

**Challenges and Opportunities:**

While these methods offer improvements, the text highlights the need for a more practical and efficient compressive memory technique. This technique should balance simplicity with the ability to maintain the quality of the LLM's output.

**Overall, the quest for effectively managing memory and context window size remains an active area of research in the LLM domain.**
